# Coleta de Dados do Sentinel-2 usando Google Earth Engine

Este notebook demonstra como coletar dados do Sentinel-2 para áreas de interesse definidas em arquivos KML.

In [1]:
import sys
sys.path.append('..')

import ee
from pathlib import Path
from src.data.gee_collector import SentinelCollector
from src.data.geometry_handler import GeometryHandler
from src.data.dataset_manager import DatasetManager

# Inicializar Earth Engine
ee.Initialize()

## 1. Configurar o Ambiente

Primeiro, vamos configurar os diretórios e inicializar os componentes necessários:

In [2]:
# Configurar diretórios
data_dir = Path('../data')
kml_dir = data_dir / 'kmls'  # Diretório único para todos os KMLs
output_dir = data_dir / 'tiffs'

# Criar diretórios se não existirem
kml_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)

# # Inicializar componentes
dataset_manager = DatasetManager(train_val_dir=str(kml_dir))
collector = SentinelCollector(
    output_dir=str(output_dir),
    geometry_handler=GeometryHandler()
)

TypeError: SentinelCollector.__init__() got an unexpected keyword argument 'output_dir'

## 2. Carregar e Dividir Regiões

O DatasetManager irá carregar todos os KMLs do diretório e dividi-los automaticamente em conjuntos de treino e validação:

In [4]:
dataset_manager.load_train_val_regions(val_split=0.2)

EEException: Invalid GeoJSON geometry.

In [ ]:
# Carregar e dividir regiões
train_regions, val_regions = dataset_manager.load_train_val_regions(val_split=0.2)

print(f"Número de regiões para treino: {len(train_regions)}")
print(f"Número de regiões para validação: {len(val_regions)}")

## 3. Coletar Imagens do Sentinel-2

Agora vamos coletar as imagens para as regiões de treino e validação:

In [ ]:
# Parâmetros de coleta
collection_params = {
    'years_back': 5,  # Últimos 2 anos de dados
    'max_cloud_cover': 20  # Máximo de 20% de cobertura de nuvens
}

# Coletar imagens
train_collection = collector.get_sentinel_collection(
    regions=train_regions,
    **collection_params
)

val_collection = collector.get_sentinel_collection(
    regions=val_regions,
    **collection_params
)

## 4. Exportar para GeoTIFF

Vamos exportar as imagens coletadas para arquivos GeoTIFF:

In [ ]:
# Configurar parâmetros de exportação
export_params = {
    'scale': 10,  # Resolução de 10 metros
    'patch_size': 64,  # Tamanho do patch em pixels
    'bands': ['B8', 'B4', 'B11']  # Bandas relevantes para vegetação
}

# Exportar dados de treino
collector.export_to_drive(
    collection=train_collection,
    regions=train_regions,
    output_prefix='treino',
    **export_params
)

# Exportar dados de validação
collector.export_to_drive(
    collection=val_collection,
    regions=val_regions,
    output_prefix='validacao',
    **export_params
)

## 5. Monitorar Exportação e Próximos Passos

### Monitoramento
As tarefas de exportação serão executadas no Google Earth Engine. Você pode monitorar o progresso em:
https://code.earthengine.google.com/tasks

### Organização dos KMLs
- Coloque todos os seus arquivos KML no diretório `kmls`
- O DatasetManager irá automaticamente:
  - Carregar todos os KMLs do diretório
  - Dividir as regiões em treino/validação
  - Manter a consistência entre as divisões

### Após a Exportação
1. Baixe os arquivos GeoTIFF do Google Drive
2. Mova-os para o diretório `tiffs` do projeto
3. Use o TiffLoader para preparar os dados para treinamento

### Importante
Certifique-se de que você tem:
- Acesso configurado ao Google Earth Engine
- Espaço suficiente no Google Drive
- KMLs bem definidos para suas áreas de interesse

In [1]:
import geopandas as gpd
import numpy as np

shapefile = gpd.read_file("/home/luisantolin/Downloads/Sigef Brasil/Sigef Brasil.shp")

shapefile = shapefile.to_crs(4326)
shapefile['area_size'] = shapefile['geometry'].apply(lambda x: x.area)
shapefile = shapefile[shapefile['area_size']<= np.percentile(shapefile['area_size'],75)]


In [2]:
import geobr

states = geobr.read_state()

In [3]:
states = states.to_crs(4326)


In [4]:
shps = shapefile.sample(n=1000,weights = 'uf_id').clip(states[states.abbrev_state.isin(['RS','SC','PR','GO','MT','MS','SP'])])



In [9]:
shps['geometry'] = shps.geometry.force_2d()

In [11]:
import os

for s,i in shps.iterrows():
    shps.loc[[s]].to_file(os.path.join('/home/luisantolin/code/agri_remote_classification/data/kmls',str(i.parcela_co)+'.kml'),driver = 'KML')

/home/luisantolin/code/agri_remote_classification/.rmt/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Layer name '495ceb4b-7790-4004-a500-80ea3de32c37' adjusted to '495ceb4b_7790_4004_a500_80ea3de32c37' for XML validity.
  ogr_write(
/home/luisantolin/code/agri_remote_classification/.rmt/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Layer name 'da50ac8e-4a49-4221-894b-d43acc4485ae' adjusted to 'da50ac8e_4a49_4221_894b_d43acc4485ae' for XML validity.
  ogr_write(
/home/luisantolin/code/agri_remote_classification/.rmt/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Layer name '87846e83-83a8-47e7-838d-4246e0c5acee' adjusted to '87846e83_83a8_47e7_838d_4246e0c5acee' for XML validity.
  ogr_write(
/home/luisantolin/code/agri_remote_classification/.rmt/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Layer name '7b6d9076-7ac3-4fb5-ac2d-5e54dd76a775' adjusted to '7b6d9076_7ac3_4fb5_ac2d_5e54dd76a775' for XML validity.
  ogr